In [2]:
import pandas as pd

from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [12]:
CHROMEDRIVER_PATH = './../chromedriver'
URL = 'https://www.forbes.com/global2000/'

# Prepare DataFrame
df_list = []

# Init browser instance
browser = webdriver.Chrome(CHROMEDRIVER_PATH)

# Get page
browser.get(URL)
# Reload page
browser.refresh();

delay = 3 # seconds
pages_count = 5
parse_timeout_max = 10
df = pd.DataFrame()
try:
    last_id = 0
    for page_i in range (pages_count):
        # Wait table loading
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//table//tr//td')))
        # Get table root
        table_element = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//table')))    

        # Parse table
        table_html_string = table_element.get_attribute('outerHTML')

        # Parse table untill timeout or new data
        parse_timeout = parse_timeout_max
        
        while df.empty or (last_id == df.iloc[-1]['Rank']):
            # Parse table
            df = pd.read_html(table_html_string)[0]
            
            # Check timeout
            parse_timeout -= 1
            if parse_timeout == 0:
                raise "No new data for parsing."
                
            # Wait reloading
            sleep(1)
        
        print(last_id)
        # Save new id
        last_id = df.iloc[-1]['Rank']
        
        # Add parsed table
        df_list.append(pd.read_html(table_html_string)[0])
        
        # Click NEXT button
        next_btn = browser.find_element_by_xpath("//a[@ng-switch-when='next']").click()
        
        
except TimeoutException:
    print("Loading took too much time!")

# Close browser
browser.quit()

len(df_list)

0
100.0
200.0
300.0
400.0


5

In [13]:
# Concat
df = pd.concat(df_list)
# Clear
df = df.dropna(axis=1, how='all').dropna(axis=0, how='all')
df

,Rank,Company,Country/Territory,Sales,Profits,Assets,Market Value,Unnamed: 7
1,1,ICBC,China,$177.2 B,$45.3 B,"$4,322.5 B",$242.3 B,NaN
2,2,China Construction Bank,China,$162.1 B,$38.9 B,"$3,822 B",$203.8 B,NaN
3,3,JPMorgan Chase,United States,$142.9 B,$30 B,"$3,139.4 B",$291.7 B,NaN
4,4,Berkshire Hathaway,United States,$254.6 B,$81.4 B,$817.7 B,$455.4 B,NaN
5,5,Agricultural Bank of China,China,$148.7 B,$30.9 B,"$3,697.5 B",$147.2 B,NaN
...,...,...,...,...,...,...,...,...
102,396,National Commercial Bank,Saudi Arabia,$6.8 B,$3.1 B,$135.2 B,$29.6 B,NaN
103,397,Adidas,Germany,$25 B,$1.5 B,$22.8 B,$44.9 B,NaN
104,398,CTBC Financial,Taiwan,$14.1 B,$1.4 B,$207.5 B,$13.1 B,NaN
105,398,Danske Bank,Denmark,$16 B,$1.5 B,$589.3 B,$10.1 B,NaN


In [49]:
# Save to parquet
df.to_parquet("../data/companies_rank.parquet")